# Lab 5: Song Classification, Part 2

Welcome to Lab 5! We'll pick off where we left off and continue to build a song classifer using k-nearest neighbors. 

Please complete lab 4 before starting lab 5.

Lab 4 is part 1 of the investigation. Lab 5 is part 2 of the investigation.

You will build a classifier that guesses whether a song is hip-hop or country, using only the number of times that each word appears in a song's lyrics.  By the end of the project, you should know how to:

1. Clean and organize a dataset used to test a machine learning model
2. Build a k-nearest neighbors classifier
3. Test a classifier on data

**Advice.** Develop your answers incrementally. To perform a complicated table manipulation, break it up into steps, perform each step on a different line, give a new name to each result, and check that each intermediate result is what you expect. You can add any additional names or functions you want to the provided cells. 

To get started, load `datascience`, `numpy`, `plots`, and `okgrade`.

In [1]:
# Run this cell to set up the notebook, but please don't change it.
import numpy as np
import math
from datascience import *

# These lines set up the plotting functionality and formatting.
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter('ignore', UserWarning)

# These lines load the tests.
from okgrade import grade

## Overview: Recap

In lab 4, we completed the following tasks:
1. In section 1, we explored the dataset and split the dataset into training data and test data.
2. In section 2, we ran through a guided example of the k-Nearest Neightbors (k-NN) classification algorithm.

**If you do not remember lab 4, we highly recommend you go back and review it now. It will help you for this lab. **

In lab 5, we are going to complete the following tasks:
1. Identify some features.
2. Define a classifier function using your features and the training set.
3. Evaluate its performance (the proportion of correct classifications) on the test set.

Run the cell below to set up the lab.

In [2]:
lyrics = Table.read_table('lyrics.csv')

training_proportion = 11/16

num_songs = lyrics.num_rows
num_train = int(num_songs * training_proportion)
num_valid = num_songs - num_train

train_lyrics = lyrics.take(np.arange(num_train))
test_lyrics = lyrics.take(np.arange(num_train, num_songs))

def most_common(label, table):
    return table.group(label).sort('count', descending=True).column(label).item(0)

## 1. Features

Now, we're going to extend our classifier from lab 4 to consider more than two features at a time.

Euclidean distance still makes sense with more than two features. For `n` different features, we compute the difference between corresponding feature values for two songs, square each of the `n`  differences, sum up the resulting numbers, and take the square root of the sum.

** Question 1.1 ** <br/>
Write a function to compute the Euclidean distance between two **arrays** of features of *arbitrary* (but equal) length.  Use it to compute the distance between the first song in the training set and the first song in the test set, *using all of the features*.  (Remember that the title, artist, and genre of the songs are not features.)

**Note:** To convert row objects to arrays, use `np.array`. For example, if `t` was a table, `np.array(t.row(0))` converts row 0 of `t` into an array.

In [3]:
train_lyrics.drop(["Title", "Artist", "Genre"]).take(0)

i,the,you,to,and,a,me,it,not,in,my,is,of,your,that,do,on,are,we,am,will,all,for,no,be,have,love,so,know,this,but,with,what,just,when,like,now,que,time,can,come,de,there,go,up,oh,la,one,they,out,down,get,she,was,see,if,got,never,from,he,feel,want,let,make,way,say,take,would,as,ca,day,at,babi,away,life,yeah,y,back,by,her,heart,here,how,could,night,need,our,look,where,en,eye,thing,world,more,caus,gonna,die,right,been,tell,think,un,who,el,through,man,live,again,give,too,onli,te,tri,tu,or,whi,se,keep,dream,well,mind,an,wo,still,us,his,long,mi,girl,wanna,find,fall,around,good,about,some,over,littl,turn,hand,then,noth,light,said,call,everi,alway,were,did,into,e,wait,leav,home,had,hold,run,es,walk,ever,them,lie,face,end,hear,gone,head,believ,everyth,ya,has,cri,alon,yo,hey,their,le,someth,si,insid,du,befor,o,word,stay,place,much,chang,ich,god,stand,these,better,last,friend,soul,anoth,je,left,new,him,than,stop,tonight,burn,sun,off,por,lo,play,thought,sing,show,lost,peopl,made,break,und,start,should,realli,chorus,hard,sky,old,da,even,boy,pleas,song,care,without,hope,watch,pain,amor,move,wrong,put,free,gotta,name,et,same,blue,con,danc,talk,seem,own,mine,dark,na,far,tear,true,fire,use,dead,bring,other,sin,someon,myself,forev,close,sleep,rain,those,must,una,les,sweet,mean,fight,rememb,star,while,kiss,kill,high,breath,doe,yes,al,wish,che,real,smile,two,cold,lord,year,open,il,fear,di,done,side,fli,black,line,rock,blood,lose,found,onc,non,door,bad,ooh,wonder,ti,como,enough,para,sound,togeth,mayb,los,help,shine,mani,heaven,becaus,work,touch,behind,yourself,today,big,might,death,hate,miss,ride,street,came,ask,hide,everybodi,save,till,may,beauti,roll,sure,deep,arm,son,wind,bodi,forget,happi,understand,ai,best,ma,town,war,ah,listen,seen,der,sometim,lone,hell,morn,until,set,hurt,told,first,knew,goe,round,em,alright,music,sea,truth,qui,beat,nicht,eu,las,todo,pass,under,del,late,after,ani,ground,fool,pas,broken,goodby,das,des,kind,wall,angel,est,road,reason,dan,money,chanc,quiero,mai,part,matter,learn,els,rise,pour,saw,grow,ha,sit,land,fade,air,took,nobodi,ne,fill,easi,game,follow,wake,anyth,aliv,solo,water,heard,though,white,vida,ist,voic,woman,blind,each,citi,young,lay,moon,reach,readi,fine,soon,along,à,crazi,past,red,room,such,ein,step,scream,tomorrow,control,memori,strong,hit,everyon,wast,não,men,earth,s,upon,final,shake,im,abov,ja,moment,rest,guess,hous,born,somebodi,feet,mas,sad,laugh,esta,happen,went,cest,blow,noch,meet,whole,lover,drive,car,lead,su,veri,ni,promis,bed,apart,um,empti,shadow,full,begin,pull,child,pretti,drink,pray,king,cut,t,power,gave,hay,peac,hang,next,ser,mother,é,cuando,between,pay,carri,tire,speak,lot,tree,den,skin,tout,til,slow,sinc,anymor,kid,clear,mich,jesus,differ,fast,becom,va,gun,nos,zu,más,river,faith,floor,cross,ladi,near,fun,ring,answer,pero,afraid,jag,repeat,great,stori,yet,cannot,hot,wir,nada,hour,summer,stone,bleed,sign,mir,lyric,children,trust,wild,felt,troubl,honey,sick,perfect,human,bright,bien,mama,dich,parti,train,plan,porqu,sorri,plus,hair,secret,silenc,tast,brother,throw,within,per,push,m,somewher,three,cloud,shall,vez,low,window,knee,lip,futur,dear,search,2,sie,drop,det,cool,flame,outsid,worri,bout,goin,met,stare,sight,most,mon,voy,across,wear,wave,tus,pick,er,send,siempr,wie,bone,flow,piec,soy,uh,thank,mis,straight,blame,catch,au,ce,tight,&,mile,holi,darl,auf,mouth,christma,which,moi,against,mad,both,share,caught,father,read,warm,wing,nunca,belong,damn,eat,win,sens,comm,u,fell,evil,dem,ho,pictur,second,nowher,dog,rule,une,mal,mountain,joy,mit,top,tan,som,bit,cover,point,thousand,hoy,n,gold,bell,è,devil,bird,sur,quit,brain,return,build,kick,rose,babe,guy,whoa,shame,mein,space,whisper,tiempo,cos,write,dir,tengo,count,slip,ver,desir,sei,nice,weak,rais,jai,ocean,number,drown,spirit,scare,strang,ik,half,known,pretend,worth,someday,act,beyond,fair,million,finger,question,doubt,meant,smoke,comin,flesh,dawn,daddi,storm,hole,color,jump,remain,mundo,etern,nothin,buy,nur,anyon,dress,1,minut,dont,pra,snow,d,dig,est

In [4]:
def distance(features1, features2):
    """The Euclidean distance between two arrays of feature values."""
    return np.sqrt(np.sum(np.subtract(features1 ,features2) ** 2))

train_row0 = train_lyrics.drop(["Title", "Artist", "Genre"]).row(0)
test_row0 = test_lyrics.drop(["Title", "Artist", "Genre"]).row(0)

distance_first_to_first = distance(np.array(train_row0) , np.array(test_row0))
distance_first_to_first

0.1482277008140451

In [5]:
grade("tests/q1_1.py")

### 1.1. Creating your own feature set

Unfortunately, using all of the features has some downsides.  One clear downside is *computational* -- computing Euclidean distances just takes a long time when we have lots of features.  You might have noticed that in the last question!

So we're going to select just 20.  We'd like to choose features that are very *discriminative*. That is, features which lead us to correctly classify as much of the test set as possible.  This process of choosing features that will make a classifier work well is sometimes called *feature selection*, or more broadly *feature engineering*.

** Question 1.1.1 ** <br/>
Look through the list of features (the labels of the `lyrics` table after the first three).  Choose 20 common words that you think might let you distinguish between country and hip-hop songs. Make sure to choose words that are frequent enough that every song contains at least one of them. Don't just choose the 20 most frequent, though... you can do much better.

You might want to come back to this question later to improve your list, once you've seen how to evaluate your classifier.  The first time you answer this question, spend some time looking through the features, but not more than 15 minutes.

#### This below cell is just my analysis attempts to programmatically find the most commonly occuring words for each genre.

In [6]:
country_songs = lyrics.where("Genre", "Country").drop(["Title", "Artist", "Genre"])
country_word_frequency = []
country_words = []
for label in country_songs.labels:
    country_words.append(label)
    word_count = np.sum(country_songs.column(label))
    country_word_frequency.append(word_count)

country_table = Table().with_columns("Word",country_words, "Total Frequency", country_word_frequency )
country_most_common = country_table.sort("Total Frequency", descending = True).take(range(20))
country_most_common

Word,Total Frequency
i,43.6025
the,34.2235
you,30.8772
and,22.6695
to,19.9147
a,19.1077
not,15.3036
me,14.3102
it,13.9881
my,11.8676


In [7]:
# Set my_20_features to an array of 20 features (strings that are column labels)

my_20_features = country_most_common.column("Word")

train_20 = train_lyrics.select(my_20_features)
test_20 = test_lyrics.select(my_20_features)

This test makes sure that you have chosen words such that at least one appears in each song. If you can't find words that satisfy this test just through intuition, try writing code to print out the titles of songs that do not contain any words from your list, then look at the words they do contain.

In [8]:
grade("tests/q1_1_1.py")

Next, let's classify the first song from our test set using these features.  You can examine the song by running the cells below. Do you think it will be classified correctly?

In [9]:
print("Song:")
test_lyrics.take(0).select('Title', 'Artist', 'Genre').show()
print("Features:")
test_20.take(0).show()

Song:


Title,Artist,Genre
That Kind of Love,Alison Krauss,Country


Features:


i,the,you,and,to,a,not,me,it,my,in,is,that,of,do,on,your,will,love,have
0.010582,0.037037,0.005291,0.015873,0.010582,0.015873,0.026455,0.005291,0.037037,0.015873,0.005291,0.031746,0.026455,0.026455,0.005291,0.010582,0,0.005291,0.026455,0


As before, we want to look for the songs in the training set that are most alike our test song.  We will calculate the Euclidean distances from the test song (using the 20 selected features) to all songs in the training set.  You could do this with a `for` loop, but to make it computationally faster, we have provided a function, `fast_distances`, to do this for you.  Read its documentation to make sure you understand what it does.  (You don't need to read the code in its body unless you want to.)

In [10]:
# Just run this cell to define fast_distances.

def fast_distances(test_row, train_rows):
    """An array of the distances between test_row and each row in train_rows.

    Takes 2 arguments:
      test_row: A row of a table containing features of one
        test song (e.g., test_20.row(0)).
      train_rows: A table of features (for example, the whole
        table train_20)."""
    assert train_rows.num_columns < 50, "Make sure you're not using all the features of the lyrics table."
    counts_matrix = np.asmatrix(train_rows.columns).transpose()
    diff = np.tile(np.array(test_row), [counts_matrix.shape[0], 1]) - counts_matrix
    distances = np.squeeze(np.asarray(np.sqrt(np.square(diff).sum(1))))
    return distances

** Question 1.1.2 ** <br/>
Use the `fast_distances` function provided above to compute the distance from the first song in the test set to all the songs in the training set, **using your set of 20 features**.  Make a new table called `genre_and_distances` with one row for each song in the training set and two columns:
* The `"Genre"` of the training song
* The `"Distance"` from the first song in the test set 

Ensure that `genre_and_distances` is **sorted in increasing order by distance to the first test song**.

In [11]:
# The staff solution took 4 lines of code.
train_genres = train_lyrics.column("Genre")
train_distance_from_Test0 = fast_distances(test_20.row(0), train_20)
genre_and_distances = Table().with_columns("Genre",train_genres, "Distance", train_distance_from_Test0).sort("Distance")
genre_and_distances

Genre,Distance
Hip-hop,0.0504493
Hip-hop,0.0509764
Hip-hop,0.0522303
Country,0.0528064
Country,0.0528843
Hip-hop,0.0534353
Country,0.0541689
Hip-hop,0.0541787
Hip-hop,0.054431
Hip-hop,0.0546424


In [12]:
grade("tests/q1_1_2.py")

** Question 1.1.3 ** <br/>
Now compute the 5-nearest neighbors classification of the first song in the test set.  That is, decide on its genre by finding the most common genre among its 5 nearest neighbors, according to the distances you've calculated.  Then check whether your classifier chose the right genre.  (Depending on the features you chose, your classifier might not get this song right, and that's okay.)

In [13]:
# Set my_assigned_genre to the most common genre among these.
my_assigned_genre = genre_and_distances.take(range(5)).group("Genre").sort("count", descending = True).column("Genre").item(0)
print(my_assigned_genre)
# Set my_assigned_genre_was_correct to True if my_assigned_genre
# matches the actual genre of the first song in the test set.
test_row0_genre = test_lyrics.take(0).column("Genre").item(0)
print(test_row0_genre)
my_assigned_genre_was_correct =  my_assigned_genre == test_row0_genre 

print("The assigned genre, {}, was{}correct.".format(my_assigned_genre, " " if my_assigned_genre_was_correct else " not "))

Hip-hop
Country
The assigned genre, Hip-hop, was not correct.


In [14]:
grade("tests/q1_1_3.py")

### 1.2. A classifier function

Now we can write a single function that encapsulates the whole process of classification.

** Question 1.2.1 ** <br/>
Write a function called `classify`.  It should take the following four arguments:
* A row of features for a song to classify (e.g., `test_20.row(0)`).
* A table with a column for each feature (e.g., `train_20`).
* An array of classes that has as many items as the previous table has rows, and in the same order.
* `k`, the number of neighbors to use in classification.

It should return the class a `k`-nearest neighbor classifier picks for the given row of features (the string `'Country'` or the string `'Hip-hop'`).

In [15]:
def classify(test_row, train_rows, train_classes, k):
    """Return the most common class among k nearest neigbors to test_row."""
    distances = fast_distances(test_row, train_rows)
    #train_genres = train_lyrics.column("Genre")
    genre_and_distances = Table().with_columns("Genre",train_classes, "Distance", distances).sort("Distance")
    predicted_genre = genre_and_distances.take(range(k)).group("Genre").sort("count", descending = True).column("Genre").item(0)
    #print("Predicted Genre is ", predicted_genre)
    return predicted_genre

In [16]:
grade("tests/q1_2_1.py")

** Question 1.2.2 ** <br/>
Assign `grandpa_genre` to the genre predicted by your classifier for the song "Grandpa Got Runned Over By A John Deere" in the test set, using **9 neighbors** and using your 20 features.

In [17]:
# The staff solution first defined a row object called grandpa_features.
grandpa_features = lyrics.where("Title","Grandpa Got Runned Over By A John Deere" ).select(my_20_features).row(0)
train_classes = train_lyrics.column("Genre")
grandpa_genre = classify(grandpa_features, train_20,train_classes, 9)
grandpa_genre

'Hip-hop'

In [18]:
grade("tests/q1_2_2.py")

Finally, when we evaluate our classifier, it will be useful to have a classification function that is specialized to use a fixed training set and a fixed value of `k`.

** Question 1.2.3 ** <br/>
Create a classification function that takes as its argument a row containing your 20 features and classifies that row using the 5-nearest neighbors algorithm with `train_20` as its training set.

In [19]:
train_classes

array(['Hip-hop', 'Hip-hop', 'Hip-hop', ..., 'Country', 'Hip-hop',
       'Country'], dtype='<U7')

In [20]:
def classify_one_argument(row):
    return classify(row, train_20, train_classes,5)

# When you're done, this should produce 'Hip-hop' or 'Country'.
classify_one_argument(test_20.row(0))

'Hip-hop'

In [21]:
grade("tests/q1_2_3.py")

### 1.3. Evaluating your classifier

Now that it's easy to use the classifier, let's see how accurate it is on the whole test set.

**Question 1.3.1.** Use `classify_one_argument` and `apply` to classify every song in the test set.  Name these guesses `test_guesses`.  **Then**, compute the proportion of correct classifications. 

In [22]:
test_guesses = test_lyrics.select(my_20_features).apply(classify_one_argument)
proportion_correct = np.average(test_lyrics.column("Genre") == test_guesses)
# sum(test_lyrics.column("Genre") == test_guesses)/test_lyrics.num_rows
proportion_correct

0.6412639405204461

In [23]:
grade("tests/q1_3_1.py")

At this point, you've gone through one cycle of classifier design.  Let's summarize the steps:
1. From available data, select test and training sets.
2. Choose an algorithm you're going to use for classification.
3. Identify some features.
4. Define a classifier function using your features and the training set.
5. Evaluate its performance (the proportion of correct classifications) on the test set.

## Submission

Congratulations! You're finished with lab 5 and Data 8.3x! You've created your own song classifer using k-nearest neighbors. 

**If you want to continue, you can read about classification online and try to build an even more accurate classifier.**

In order to successfully submit your assignment, follow these steps...
- **IMPORTANT** Before you do anything, **Save and Checkpoint** from the `File` menu. Please do this first before running the cell below,
- **run all the tests and verify that they all pass** (the next cell has a shortcut for that), 
- **Review the notebook one last time, we will be grading the final state of your notebook** If you make any changes, please **Save and Checkpoint** again.

In [24]:
# For your convenience, you can run this cell to run all the tests at once!
import glob
from okgrade.notebook import grade_notebook
if not globals().get('__OKGRADE__', False):
    display(grade_notebook('lab05.ipynb', sorted(glob.glob('tests/q*.py'))))